# MLflow demo

## Start MLflow server

[MLflow](https://mlflow.org) is a versatile, expandable, open-source platform for managing workflows and artifacts across the machine learning lifecycle. It has built-in integrations with many popular ML libraries, but can be used with any library, algorithm, or deployment tool. It is designed to be extensible, so you can write plugins to support new workflows, libraries, and tools.

You can easily install MLflow for your tasks with the following script:

In [ ]:
!cat manutils/start-mlflow.sh

To run install process open a terminal and type `cd ~ && __MANUAL/manutils/start-mlflow.sh` and MLflow will be installed.

## Run example

In [ ]:
import os
import mlflow
from mlflow import log_metric, log_param, log_params, log_artifacts
from mlflow.models.signature import infer_signature
from random import random, randint
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes
from sklearn.ensemble import RandomForestRegressor

mlflow.set_tracking_uri(f'file:///home/jovyan/{os.environ["JUPYTERHUB_USER"]}_mlflow')
print('MLflow UI available at:',
      'https://jhas01.gsom.spbu.ru{}proxy/{}/'.format(
          os.environ['JUPYTERHUB_SERVICE_PREFIX'], 50000))

__NOTE:__ MLflow is configured to store its artifacts in the `<YOUR_LOGIN_NAME>_mlflow` directory that will be created whrn MLflow is started.

Here is a base example how to use MLflow for your tasks:

In [ ]:
with mlflow.start_run() as run:
    # Load the diabetes dataset.
    db = load_diabetes()
    X_train, X_test, y_train, y_test = train_test_split(db.data, db.target)

    # Create and train models.
    rf = RandomForestRegressor(n_estimators=100, max_depth=6, max_features=3)
    rf.fit(X_train, y_train)

    # Use the model to make predictions on the test dataset.
    predictions = rf.predict(X_test)
    signature = infer_signature(X_test, predictions)
    mlflow.sklearn.log_model(rf, "model", signature=signature)

    print("Run ID: {}".format(run.info.run_id))